# Predict opioid prescribers with scikit-learn

Based on this [notebook](https://github.com/IBM/predict-opioid-prescribers) by Madison Myers


## Import libraries and load data

In [ ]:
from io import StringIO
import requests
import json
import pandas as pd

Load 3 datasets into the notebook by using the `Insert to code` button below for each file from the *Files* menu on the right and then click `Insert Pandas DataFrame`. Make sure that they are named correctly as these will be used the rest of the notebook:

- 'df_data_1': opioids.csv
- 'df_data_2': overdoses.csv
- 'df_data_3': prescriber-info.csv

In [ ]:
# insert file 1


In [ ]:
# insert file 2


In [ ]:
# insert file 3


## Exploration and Initial Preprocessing

Let's start by removing the `,` from the numbers in the `Deaths` and `Population` columns so that they can be used as integers.

In [ ]:
df_data_2['Deaths'] = df_data_2['Deaths'].str.replace(',', '')
df_data_2['Deaths'] = df_data_2['Deaths'].astype(int)

In [ ]:
df_data_2['Population'] = df_data_2['Population'].str.replace(',', '')
df_data_2['Population'] = df_data_2['Population'].astype(int)

Add an additional column with the deaths per capita `Deaths/Population` for each state:

In [ ]:
df_data_2['Deaths/Population'] = (df_data_2['Deaths']/df_data_2['Population'])

In [ ]:
df_data_2.head()

Use pixiedust to visualize the data.

In [ ]:
import pixiedust

### How many opioid deaths by U.S. state?

With PixieDust data can be easily displayed. Have a play and change the chart type and change the settings on the right of the chart (if you get an error just try another option). 

In [ ]:
display(df_data_2)

It definitely looks like California has a great deal more deaths than any other state. But California is a huge state with a matching population. Because of this we need to take a look at the values of deaths per capita.

### What about deaths (% of population) by U.S. State?

In [ ]:
display(df_data_2)

West Virginia, New Mexico, New Hampshire, Ohio, Kentucky and Delaware stand out. 

Let's check this out with a map.

In [ ]:
display(df_data_2)

Let's move on to exploring the other dataset.

In [ ]:
df_data_3.head()

There is a large number of prescriptions with their physicians' gender, state, speciality, whether they are an opioid prescriber or not and unique ID.

In [ ]:
df_data_3.count()

In [ ]:
#Let's take a look at the states. Why are there more than 50 states?
df_data_3.State.unique()

In [ ]:
#Compare to df_data_2.
df_data_2.Abbrev.unique()

In [ ]:
#Clean up states and make the dataset state list equal.
#I checked the list of US state abbreviations and did not recognize PR, AE, ZZ, GU, AA or VI. After checking I learned that PR is Puerto Rico, GU is Guam and VI is Virgin Islands.
#Though I identified 3 of the 6 unknowns, I'll remove all of them as dataset 2 does not have data regarding PR, GU or VI.
df_data_3 = df_data_3[df_data_3.State != 'AE']
df_data_3 = df_data_3[df_data_3.State != 'ZZ']
df_data_3 = df_data_3[df_data_3.State != 'AA']
df_data_3 = df_data_3[df_data_3.State != 'PR']
df_data_3 = df_data_3[df_data_3.State != 'GU']
df_data_3 = df_data_3[df_data_3.State != 'VI']

In [ ]:
#Make sure it worked!
df_data_3.State.unique()

In [ ]:
#Check how many credentials there are.
df_data_3.Credentials.unique()

In [ ]:
#Check out the specialties.
df_data_3.Specialty.unique()

In [ ]:
#How much of the dataset is male vs female?
df_data_3.groupby('Gender').size() / df_data_3.groupby('Gender').size().sum()

In [ ]:
#What is the percentage of prescribers in our dataset that prescribe opioid drugs vs that do not?
df_data_3.groupby('Opioid.Prescriber').size() / df_data_3.groupby('Opioid.Prescriber').size().sum()

In [ ]:
#Plot the opioid prescriber count vs non opioid prescriber count.
#The dataset has a slightly higher number of opioid prescribers.
pd.value_counts(df_data_3['Opioid.Prescriber']).plot.bar()

## Creating our Classifiers to Predict Opioid Prescribers

Can we predict who prescribed opioids based on this data? 

## Combination of Approaches: Kaggle and Indiana University

 - "Quick and Dirty" approach from Kaggle (https://www.kaggle.com/jiashenliu/quick-and-dirty-attempt-on-voting-classifier)
 - "Detecting Frequent Opioid Prescription" (https://inclass.kaggle.com/apryor6/detecting-frequent-opioid-prescription)
 - Indiana University: "Opiate prescription analysis using machine learning" (http://cgi.soic.indiana.edu/~arunsank/AML_report.pdf)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split

Find the shape of the data frame to set up our classifiers:

In [ ]:
print(df_data_3.shape)

Mark opioid vs non opiod drugs in df_data_3 with use of the data in df_data_1:

In [ ]:
opioids = df_data_1 
name=opioids['Drug Name']
import re
new_name=name.apply(lambda x:re.sub("\ |-",".",str(x)))
columns=df_data_3.columns
Abandoned_variables = set(columns).intersection(set(new_name))
Kept_variable=[]
for each in columns:
    if each in Abandoned_variables:
        pass
    else:
        Kept_variable.append(each)

Check the new shape:

In [ ]:
df=df_data_3[Kept_variable]
print(df.shape)

In [ ]:
df.head()

Remove the Credentials and NPI column to trim the features down.

In [ ]:
df = df.drop(df.columns[[0, 3]], axis=1) 
df.head()

Create training and test data:

In [ ]:
train,test = train_test_split(df,test_size=0.2,random_state=42)
print(train.shape)
print(test.shape)

Convert the categorical columns:

In [ ]:
Categorical_columns=['Gender','State','Specialty']

for col in Categorical_columns:
    train[col]=pd.factorize(train[col], sort=True)[0]
    test[col] =pd.factorize(test[col],sort=True)[0]

Set the features:

In [ ]:
features=train.iloc[:,1:242] #make sure we only use the columns that we want as our features
features.head()

In [ ]:
import sklearn
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

Train the models. Use several classifiers to check which one has the highest accuracy.

Some are commented out as they take a bit too long to run for this lab. Be patient as it will take a couple of minutes to optimize the models.

In [ ]:
features=train.iloc[:,0:242] #Make sure to remove Opioid.Prescriber (the target)!
target = train['Opioid.Prescriber']
Name=[]
Accuracy=[]
model1=LogisticRegression(random_state=22,C=0.000000001,solver='liblinear',max_iter=200)
model2=GaussianNB()
model3=RandomForestClassifier(n_estimators=200,random_state=22)
model4=GradientBoostingClassifier(n_estimators=200)
model5=KNeighborsClassifier()
model6=DecisionTreeClassifier()
model7=LinearDiscriminantAnalysis()
Ensembled_model=VotingClassifier(estimators=[('lr', model1), ('gn', model2), ('rf', model3),('gb',model4),('kn',model5),('dt',model6),('lda',model7)], voting='hard')
#for model, label in zip([model1, model2, model3, model4,model5,model6,model7], ['Logistic Regression','Naive Bayes','Random Forest', 'Gradient Boosting','KNN','Decision Tree','LDA']):
for model, label in zip([model1, model3, model4 ,model6], ['Logistic Regression','Random Forest', 'Gradient Boosting','Decision Tree']):
    scores = cross_val_score(model, features, target, cv=5, scoring='accuracy')
    Accuracy.append(scores.mean())
    Name.append(model.__class__.__name__)
    print("Accuracy: %f of model %s" % (scores.mean(),label))

### Evaluate

Model 3, 4 and 6 perform best. Let's predict the opioids prescriber for each of them with the test data with `fit()` and calculate the accuracy

In [ ]:
from sklearn.metrics import accuracy_score
classifers=[model3,model4,model6]
out_sample_accuracy=[]
Name_2=[]
for each in classifers:
    fit=each.fit(features,target)
    pred=fit.predict(test.iloc[:,0:242])
    accuracy=accuracy_score(test['Opioid.Prescriber'],pred)
    Name_2.append(each.__class__.__name__)
    out_sample_accuracy.append(accuracy)

In [ ]:
Name_2

In [ ]:
out_sample_accuracy

### The confusion matrix

You can read this when you keep in mind that the true label is on the 'y-axis' and the predicted label on the 'x-axis'. This means the top left and lower left are the numbers that are predicted correctly, and the bottom left and top right are predicted wrong.

In [ ]:
from sklearn.metrics import confusion_matrix
y_actu = test['Opioid.Prescriber']
confusion_matrix(y_actu, pred)

After running various classifiers, we find that Random Forest and Gradient Boostinghad the best performance comparatively. This means that if we were to build a larger project, we could focus on these particular classifiers, building upon them to help predict opioid prescribers (given more years of data). 